In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert
from torch.utils.data import TensorDataset


from project_functions import *
from project_objects import *
from project_models import *
from project_constants import DEVICE as device
from project_constants import SEED

In [ ]:
torch.manual_seed(SEED)
torch.set_default_dtype(torch.double)

## 2 Object Localization
#### First we load and inspect the localization datasets

In [ ]:
loc_train = torch.load('data/localization_train.pt')
loc_val = torch.load('data/localization_val.pt')
loc_test = torch.load('data/localization_test.pt')

In [ ]:
print(f'Train data size: {len(loc_train)}')
print(f'Val data size: {len(loc_val)}')
print(f'Test data size: {len(loc_test)}')

In [ ]:
first_img, first_label = loc_train[0]

print(f'Shape of first image: {first_img.shape}')
print(f'Type of first image: {type(first_img)}')

print(f'\nShape of first label: {first_label.shape}')
print(f'Type of first label: {type(first_label)})')
first_label

In [ ]:
# Assuming train_data, val_data, and test_data are defined elsewhere
count_instances(loc_train, 'Training Data')
count_instances(loc_val, 'Validation Data')
count_instances(loc_test, 'Test Data')


#### Plotting one image from each class

In [ ]:
plot_images(loc_train)

In [ ]:
plot_class(loc_train, 4, 0)

#### Defining a normalizer and a preprocessor

In [ ]:
imgs = torch.stack([img for img, _ in loc_train])

# Define normalizer
normalizer_pipe = transforms.Normalize(
    imgs.mean(dim=(0, 2, 3)), 
    imgs.std(dim=(0, 2, 3))
    )

# Define preprocessor including the normalizer
preprocessor = transforms.Compose([
            normalizer_pipe
        ])

In [ ]:
loc_train_norm = [(preprocessor(img), label) for img, label in loc_train]
loc_val_norm = [(preprocessor(img), label) for img, label in loc_val]
loc_test_norm = [(preprocessor(img), label) for img, label in loc_test]

In [ ]:
train_loader = torch.utils.data.DataLoader(loc_train_norm, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(loc_val_norm, batch_size=64, shuffle=False)

loss_fn = LocalizationLoss()

#### Defining models

In [ ]:
model_name = 'blejh'

torch.manual_seed(SEED)
model = MyCNN1((48,60,1))
model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

loss_train, loss_val, train_perform, val_perform, losses_separated = train(
    n_epochs=1,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn,
    train_loader=train_loader,
    val_loader=val_loader
    performance=compute_performance()
)

plot_loss(loss_train, loss_val, model_name, save_model=True)
plot_lists(losses_separated, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)
y_true, y_pred = predict(model, val_loader)
plot_predictions(loc_val, y_true, y_pred, label=4, start_idx=0, fig_name=model_name, save_model=False)

#### Selecting the best model

In [ ]:
best_model, best_performance = model_selector([model], [val_perform])

# Print additional details of the best model
print("Best Model Details\n--------------------------------------------------------------")
print(f"Network architecture/ layout: {best_model}\n")
#print(f"Optimizer Parameters: {best_data.optimizer")
print(f"Validation Performance: {best_performance}")
#print(f"Validation Accuracy {round(best_data['model_man_val_accuracy'], 2)}")

#### Evaluating the best model on unseen data TBD

In [ ]:
test_loader = torch.utils.data.DataLoader(loc_test_norm, batch_size=64, shuffle=False)

test_acc, test_iou, test_performance = compute_performance(best_model, test_loader)
print(10*'-'+'Test Performance' + 10*'-')
print(f"Test Accuracy: {test_acc}\nTest IOU: {test_iou}\nOverall Performance: {test_performance}")

In [ ]:
y_true, y_pred = predict(best_model, test_loader)

In [ ]:
plot_predictions(loc_test, y_true, y_pred, label=6, start_idx=0)

# 3 Object Detection

#### Loading the data and inspecting the data

In [ ]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision.utils import draw_bounding_boxes
from torchvision.ops import box_convert
from torch.utils.data import TensorDataset


from project_functions import *
from project_objects import *
from project_models import *
from project_constants import DEVICE as device
from project_constants import SEED

torch.set_default_dtype(torch.double)

In [ ]:
train_labels = torch.load('data/list_y_true_train.pt')
val_labels = torch.load('data/list_y_true_val.pt')
test_labels = torch.load('data/list_y_true_test.pt')

In [ ]:
print(f'Train label size: {len(train_labels)}')
print(f'Val label size: {len(val_labels)}')
print(f'Test label size: {len(test_labels)}')

In [ ]:
train_imgs = torch.load('data/detection_train.pt')
val_imgs = torch.load('data/detection_val.pt')
test_imgs = torch.load('data/detection_test.pt')

In [ ]:
train_labels_local = prepare_labels(train_labels, (2,3,6))
val_labels_local = prepare_labels(val_labels, (2,3,6))
test_labels_local = prepare_labels(test_labels, (2,3,6))

In [ ]:
input_tensor = prepare_labels(train_labels, (2,3,6))

In [ ]:
def local_to_global_list(input_tensor):

    returned_list = []

    h_size = input_tensor.shape[1]
    w_size = input_tensor.shape[2]

    for h in range(h_size):


        for w in range(w_size):

            input_tensor[:,h,w,1] /=3
            input_tensor[:,h,w,2] /=2

            input_tensor[:,h,w,1] += (w*1)/w_size
            input_tensor[:,h,w,2] += (h*1)/h_size

            input_tensor[:,h,w,3] *=3
            input_tensor[:,h,w,4] *=2

    new_tensor = input_tensor.view(-1, input_tensor.size(-1), input_tensor.size(-1))
    mask = new_tensor[:, :, 0] != 0
    filtered_tensors = [new_tensor[i][mask[i]] for i in range(new_tensor.size(0))]

    for each in filtered_tensors:
        inner = []
        for i in each:
            inner.append(i)
        returned_list.append(inner)

    return returned_list



In [ ]:
yeh[0].view(2, yeh[0].size(-1))

In [ ]:

new_tensor = input_tensor.view(-1, input_tensor.size(-1), input_tensor.size(-1))
new_tensor[0][0]

In [ ]:
det_train_manual = merge_datasets(det_train, input_tensor)
det_train_manual

#### Plotting some images from the detection dataset

In [ ]:
imgs = [img for img,_ in det_train]
plot_detection_data(imgs, train_labels, start_idx=10)

#### Normalizing the dataset TBD

#### Defining the loss function

In [ ]:
class DetectionLoss(nn.Module):
    """Custom loss function"""
    def __init__(self):
        super().__init__()
        self.Localization_loss = LocalizationLoss()

    def forward(self, y_pred, y_true):
        #print(y_pred.shape)
        #print(y_true.shape)
        if y_true.shape != y_pred.shape:
            print(y_true.shape)
        #y_pred_transformed = y_pred.view(-1, y_pred.size(-1))
        #y_true_transformed = y_true.view(-1, y_true.size(-1))
        y_pred_reshaped = y_pred.permute(0,2,3,1)
        y_pred_reshaped = y_pred_reshaped.reshape(-1, y_pred_reshaped.size(-1))

        y_true_reshaped = y_true.permute(0,2,3,1)
        y_true_reshaped = y_true_reshaped.reshape(-1, y_true_reshaped.size(-1))
        # print(y_pred_transformed.shape)
        # print(y_pred_transformed)
        # print(y_true_transformed.shape)
        # print(y_true_transformed)

        loss, loss_tuples = self.Localization_loss(y_pred_reshaped, y_true_reshaped)

        #print(loss.item())
        
        return loss, loss_tuples

In [ ]:
train_loader = torch.utils.data.DataLoader(det_train, batch_size=64, shuffle=False)
val_loader = torch.utils.data.DataLoader(det_val, batch_size=64, shuffle=False)

loss_fn2 = DetectionLoss()

In [ ]:
class TestCNN(nn.Module):
    def __init__(self, input_size=None):
        self.input_size = input_size
        
        super().__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=16, kernel_size=3, stride=1, padding=1, device=device, dtype=torch.double)
        self.pool1 = nn.MaxPool2d(kernel_size=2)
        self.conv2 = nn.Conv2d(in_channels=16, out_channels=32, kernel_size=3, stride=1, padding=1, device=device, dtype=torch.double)
        self.pool2 = nn.MaxPool2d(kernel_size=2)
        self.conv3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,2), stride=1, padding=1, device=device, dtype=torch.double)
        self.pool3 = nn.MaxPool2d(kernel_size=2)
        self.conv4 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=5, stride=1, padding=1, device=device, dtype=torch.double)
        self.pool4 = nn.MaxPool2d(kernel_size=2)
        self.conv5 = nn.Conv2d(in_channels=128, out_channels=64, kernel_size=3, stride=1, padding=1, device=device, dtype=torch.double)
        self.conv6 = nn.Conv2d(in_channels=64, out_channels=32, kernel_size=3, stride=1, padding=1, device=device, dtype=torch.double)
        self.conv7 = nn.Conv2d(in_channels=32, out_channels=6, kernel_size=3, stride=1, padding=1, device=device, dtype=torch.double)


    def forward(self, x):
        out = F.relu(self.conv1(x))
        out = self.pool1(out)
        out = F.relu(self.conv2(out))
        out = self.pool2(out)
        out = F.relu(self.conv3(out))
        out = self.pool3(out)
        out = F.relu(self.conv4(out))
        out = self.pool4(out)
        out = F.relu(self.conv5(out))
        out = F.relu(self.conv6(out))
        out = self.conv7(out)

        return out

In [ ]:
model_name = 'test'
torch.manual_seed(SEED)
model = TestCNN()
model.to(device=device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=0.001)

loss_train, loss_val, train_perform, val_perform, losses_separated = train(
    n_epochs=2,
    optimizer=optimizer,
    model=model,
    loss_fn=loss_fn2,
    train_loader=train_loader,
    val_loader=val_loader
)

plot_loss(loss_train, loss_val, model_name, save_model=False)
plot_lists(losses_separated, ['detection loss', 'localization loss', 'classification loss'], model_name, save_model=False)

In [ ]:
y_true, y_pred = predict(model, val_loader)
plot_predictions(loc_val, y_true, y_pred, label=4, start_idx=0, fig_name=model_name, save_model=False)

In [ ]:
def calculate_ap(model, loader):
    '''
    Description
    '''
    model.eval()
    correct = 0
    total = 0
    iou_sum = 0

    with torch.inference_mode():
        for imgs, labels in loader:
            imgs = imgs.to(device=device, dtype=torch.double)
            labels = labels.to(device=device, dtype=torch.double)

            outputs = model(imgs)

            det_pred = F.sigmoid(outputs[:, 0])
            object_detected = det_pred > 0.5

            _, class_pred = torch.max(outputs[:, 5:], dim=1)

            det_true = labels[:, 0].int()
            class_true = labels[:, -1].int()

            total += labels.shape[0]
            correct += ((object_detected == 0) & (det_true == 0)).sum()
            correct += ((object_detected == 1) & (det_true == 1) & (class_pred == class_true)).sum()

            bbox_pred = outputs[:, 1:5]
            bbox_true = labels[:, 1:5]

            converted_bbox_pred = box_convert(bbox_pred[object_detected], in_fmt='cxcywh', out_fmt='xyxy')
            converted_bbox_true = box_convert(bbox_true[object_detected], in_fmt='cxcywh', out_fmt='xyxy')
            
            bbox_iou = box_iou(converted_bbox_pred,converted_bbox_true)
            mean_iou = bbox_iou.diag().sum()
            
            iou_sum += mean_iou.item()
            
    acc =  correct / total
    iou = iou_sum / total

    performance = (acc + iou) / 2
    
    return None, None, None